In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pulsar-classification-for-class-prediction/Pulsar.csv
/kaggle/input/predicting-pulsar-starintermediate/pulsar_data_train.csv
/kaggle/input/predicting-pulsar-starintermediate/pulsar_data_test.csv
/kaggle/input/playground-series-s3e10/sample_submission.csv
/kaggle/input/playground-series-s3e10/train.csv
/kaggle/input/playground-series-s3e10/test.csv


In [66]:
df_train = pd.read_csv('/kaggle/input/playground-series-s3e10/train.csv')
df_test = pd.read_csv('/kaggle/input/playground-series-s3e10/test.csv')
# back_train = pd.read_csv('/kaggle/input/pulsar-classification-for-class-prediction/Pulsar.csv')
# df_train['is_generated'] = 1
# df_test['is_generated'] = 1
# back_train['is_generated'] = 0
df_train.head(2)

,id,Mean_Integrated,SD,EK,Skewness,Mean_DMSNR_Curve,SD_DMSNR_Curve,EK_DMSNR_Curve,Skewness_DMSNR_Curve,Class
0,0,133.171875,59.716081,0.043133,-0.703383,54.917224,70.084438,0.749798,-0.649512,0
1,1,87.093750,36.257973,0.435469,2.266057,3.417224,21.865069,7.039330,52.686251,0


In [37]:
def remove_outlier(train):
    outliers = train[
        (train['Class']==1)&
        (train['Mean_Integrated']>115)&
        (train['SD']>45)&
        (train['EK']<0.03)&
        (train['Skewness']<1)&
        (train['Mean_DMSNR_Curve']<20)
    ].index
    train.drop(outliers,inplace=True)
    train.reset_index()
    return train


In [67]:
X = df_train.drop(columns=['id'])
# train_data = pd.concat([X, back_train],axis=0)
df_train = remove_outlier(df_train)
X = df_train.drop(columns=['id','Class'])
y = df_train['Class']
test = df_test.drop(columns=['id'])

In [41]:
def f_importance_plot(f_imp):
    fig = plt.figure(figsize = (15, 0.35*len(f_imp)))
    plt.title('Feature importances', size=25, y=1.05, 
              fontname='Calibri', fontweight='bold', color='#444444')
    a = sns.barplot(data=f_imp, x='avg_imp', y='feature', 
                    palette='Blues_d', linestyle="-", 
                    linewidth=1, edgecolor="black")
    plt.xlabel('')
    plt.xticks([])
    plt.ylabel('')
    plt.yticks(size=11, color='#444444')
    
    for j in ['right', 'top', 'bottom']:
        a.spines[j].set_visible(False)
    for j in ['left']:
        a.spines[j].set_linewidth(0.5)
    plt.show()

In [22]:
from itertools import combinations
from sklearn.preprocessing import StandardScaler
class Preprocessor:
    def __init__(self, numeric_columns=None, max_pattern=2):
        self.numeric_columns = numeric_columns
        self.max_pattern = max_pattern
        self.scaler = None
        
    def preprocess(self, X_train, X_test):
        
        X_train = self.create_numeric_combinations(X_train)
        X_test = self.create_numeric_combinations(X_test)
        
        numeric_columns = [_ for _ in X_train.columns if X_train[_].dtype == 'float']
        scaler = StandardScaler()
        X_train[numeric_columns] = scaler.fit_transform(X_train[numeric_columns])
        X_test[numeric_columns] = scaler.transform(X_test[numeric_columns])

        return X_train, X_test
    
    def create_numeric_combinations(self, df):
        new_cols = []
        for comb in range(2, len(self.numeric_columns) + 1):
            for col in combinations(self.numeric_columns, comb):
                if len(col) > self.max_pattern:
                    break
                col_names = list(col)
                new_col = '_'.join(col_names) + '_mult'
                df[new_col] = df[col_names[0]]
                for c in col_names[1:]:
                    df[new_col] *= df[c]
                new_cols.append(new_col)

        return df

numeric_columns = [_ for _ in test.columns if 'is_generated' not in _]
pp = Preprocessor(numeric_columns)
X, test = pp.preprocess(X, test)
print(f"X_train shape :{X.shape}", f"X_test shape :{test.shape}")

X_train shape :(117564, 36) X_test shape :(78377, 36)


In [68]:

from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

import optuna
import xgboost as xgb
import lightgbm as lgb
from sklearn.metrics import accuracy_score
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import log_loss

lgb_params = {
    'n_estimators': 150,
    'max_depth': 3,
    'learning_rate': 0.19503709130426908,
    'subsample': 0.016314517193550798,
    'colsample_bytree': 0.4430207740011624,
    'reg_alpha': 2.3355195856646724e-06,
    'reg_lambda': 0.6375339245312847,
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'random_state': 3407
}

f_imp = pd.DataFrame({'feature': X.columns})
FOLDS = 10
seed = 3407
bold = ['\033[1m', '\033[0m']
temp_pred = []
temp_y = []
logs = [] 
cv_scores = []
predictions =[] 
k = KFold(n_splits=FOLDS, random_state=seed, shuffle=True)
for fold, (train_idx, val_idx) in enumerate(k.split(X, y)):
    print(f'\n--- FOLD {fold+1} ---')
    model = lgb.LGBMClassifier(**lgb_params)
    model = model.fit(X.iloc[train_idx],y.iloc[train_idx])
    x_test,y_test = X.iloc[val_idx],y.iloc[val_idx]
    fold_xt_pred = model.predict_proba(x_test)[:,1]
    fold_test_pred = model.predict_proba(test)[:,1]
    temp_pred.append(fold_xt_pred)
    temp_y.append(y_test)
    logs.append(log_loss(y_test,fold_xt_pred))
    print("--------------", log_loss(y_test,fold_xt_pred))
    predictions.append(fold_test_pred)
    cv_scores.append(np.mean(logs))
lgbm_cv_score = np.mean(cv_scores)
print('LGBM CV SCORE : ', lgbm_cv_score)


--- FOLD 1 ---
-------------- 0.029738930568252466

--- FOLD 2 ---
-------------- 0.03465466131317763

--- FOLD 3 ---
-------------- 0.032220371827969684

--- FOLD 4 ---
-------------- 0.0357303584690915

--- FOLD 5 ---
-------------- 0.02814698066349476

--- FOLD 6 ---
-------------- 0.02778833055715742

--- FOLD 7 ---
-------------- 0.029230339971668472

--- FOLD 8 ---
-------------- 0.030885164512316286

--- FOLD 9 ---
-------------- 0.03386165016802756

--- FOLD 10 ---
-------------- 0.030494838469644032
LGBM CV SCORE :  0.03154639346568204


In [8]:
from sklearn.isotonic import IsotonicRegression
def iso_calib(valid_predictions,valid_target,test_pred):   
    isotonic_reg = IsotonicRegression(out_of_bounds='clip')
    isotonic_reg.fit(valid_predictions,valid_target)
    isotonic_pred_caliberated = isotonic_reg.predict(test_pred)
    return isotonic_pred_caliberated

In [50]:
lgbm_prediction = np.mean(predictions,axis=0)
# caliberated_predictions = list()
# for i in range(FOLDS):
#     caliberated_predictions.append(iso_calib(temp_pred[i],temp_y[i],lgbm_prediction))
# lgbm_pred_test = np.mean(caliberated_predictions,axis=0)
lgbm_prediction

array([3.03999909e-05, 1.62661084e-03, 4.08172523e-05, ...,
       7.43001612e-05, 7.17676297e-02, 9.76153176e-01])

In [48]:
predictions

[array([2.24037300e-05, 4.83338892e-04, 3.45209506e-05, ...,
        6.33078336e-05, 7.69813631e-02, 9.71103753e-01]),
 array([2.04518749e-05, 2.23273104e-03, 4.15749072e-05, ...,
        7.06551302e-05, 8.75789220e-02, 9.74203701e-01]),
 array([2.67536003e-05, 1.15366977e-03, 4.71352974e-05, ...,
        6.29925468e-05, 6.72827768e-02, 9.62225805e-01]),
 array([1.79713748e-05, 1.87812231e-03, 2.64231712e-05, ...,
        3.06837376e-05, 7.48136387e-02, 9.83999044e-01]),
 array([2.35229316e-05, 5.04682255e-04, 3.67686256e-05, ...,
        8.52701307e-05, 9.65946636e-02, 9.74182961e-01]),
 array([4.73831261e-05, 2.07791690e-03, 2.82169831e-05, ...,
        8.00140832e-05, 7.60668660e-02, 9.82773951e-01]),
 array([1.82300066e-05, 2.16972391e-03, 4.78697118e-05, ...,
        8.43211683e-05, 5.83614087e-02, 9.85187872e-01]),
 array([1.96227656e-05, 1.49930406e-03, 3.03494742e-05, ...,
        7.60731610e-05, 9.06179450e-02, 9.71915243e-01]),
 array([8.62008823e-05, 1.49802606e-03, 5.925027

In [51]:
ans_lgb = pd.read_csv("/kaggle/input/playground-series-s3e10/sample_submission.csv")
ans_lgb["Class"] = lgbm_prediction #np.where(predictions>0.5, 1, 0)
ans_lgb.to_csv("ans_lgb.csv", index=False)

In [40]:
ans_lgb

,id,Class
0,117564,0.000108
1,117565,0.000765
2,117566,0.000120
3,117567,0.048380
4,117568,0.001903
...,...,...
78372,195936,0.000236
78373,195937,0.000162
78374,195938,0.000159
78375,195939,0.069267
